# Probabilities through monte carlo simulations

Here, rather than training a model to predict the $E[\text{dice roll score}]$, I just build a monte carlo simulation. 

In [1]:
import sys
sys.path.insert(0, '../../src')

In [2]:
import random
from collections import defaultdict

from farkle.logic import gameobjects as go

In [3]:
dh = go.DiceHand(2,3)
dh.possible_scores()

[]

In [16]:
for i in range(1, 7):
    print(i)

1
2
3
4
5
6


In [32]:
class xyz(object):
    def __init__(self, x):
        self.x = x
        self.a = asdf(self.x)
        
    def print_x(self):
        print(self.x)
        
    class asdf(object):
        def __init__(self2, y):
            self2.y = y
            
        def print_y(self2):
            print(self2.y)

In [28]:
a = xyz(5)

In [29]:
a.print_x()

5


In [30]:
a.a.print_y()

AttributeError: 'asdf' object has no attribute 'print_y'

In [48]:
class MonteCarloSimulation(object):
    def __init__(self, max_dice: int = 6):
        self.max_dice = max_dice
        dice_range = range(1, self.max_dice + 1)
        self.dice_hands = {i: go.DiceHand(num_dice=i) for i in dice_range}
        
        self.possible_score_frequencies = {i: defaultdict(int) for i in dice_range}
        self.roll_obs = {i: 0 for i in dice_range}
        self.possible_score_obs = {i: 0 for i in dice_range}
        
    def roll(self, num_dice: int):
        self.dice_hands[num_dice].roll()
        self.roll_obs[num_dice] += 1
        if self.dice_hands[num_dice].possible_scores():
            for dh in self.dice_hands[num_dice].possible_scores():
                dh: go.DiceHand
                self.possible_score_obs[num_dice] += 1
                self.possible_score_frequencies[num_dice][dh] += 1
        else:
            self.possible_score_frequencies[num_dice][None] += 1
    
    def P_farkled(self, num_dice: int):
        return self.possible_score_frequencies[num_dice][None] / self.roll_obs[num_dice]
    
    def E_score(self, num_dice: int):
        E_score = 0
        for dh, freq in self.possible_score_frequencies[num_dice].items():
            if dh: E_score += dh.score * freq
        return (E_score / self.possible_score_obs[num_dice]) * (1 - self.P_farkled(num_dice))

In [49]:
mcs = MonteCarloSimulation()

In [50]:
mcs.dice_hands[3]

DiceHand(5, 3, 1, score=0)

In [51]:
mcs.possible_score_frequencies[3]

defaultdict(int, {})

In [52]:
mcs.roll(3)

In [53]:
mcs.possible_score_frequencies[3]

defaultdict(int,
            {DiceHand(1, score=100): 1,
             DiceHand(1, 5, score=150): 1,
             DiceHand(5, score=50): 1})

In [54]:
for i in range(4):
    mcs.roll(3)

In [55]:
mcs.possible_score_frequencies[3]

defaultdict(int,
            {DiceHand(1, score=100): 2,
             DiceHand(1, 5, score=150): 2,
             DiceHand(5, score=50): 2,
             None: 2,
             DiceHand(3, 3, 3, score=300): 1})

In [4]:
class MonteCarloDiceHand(go.DiceHand):
    def __init__(self, num_dice):
        super().__init__(num_dice=num_dice)

        self.possible_score_frequencies = defaultdict(int)
        self.roll_obs = 0
        self.possible_score_obs = 0

    def roll(self):
        super().roll()
        self.roll_obs += 1
        # save resulting possible scores if any otherwise None
        if self.possible_scores():
            for dh in self.possible_scores():
                dh: go.DiceHand
                self.possible_score_obs += 1
                self.possible_score_frequencies[dh] += 1
        else:
            self.possible_score_frequencies[None] += 1

    @property
    def E_any_score(self):
        return 1 - (self.possible_score_frequencies[None] / self.roll_obs)

    @property
    def E_score_arithmetic(self):
        E_score = 0
        for dh, freq in self.possible_score_frequencies.items():
            if dh: E_score += dh.score * freq
        return (E_score / self.possible_score_obs) * self.E_any_score

In [5]:
mcdh = MonteCarloDiceHand(3)

In [6]:
mcdh

DiceHand(4, 4, 4, score=0)

In [7]:
mcdh.roll()

In [8]:
mcdh

DiceHand(1, 2, 2, score=0)

In [9]:
mcdh.possible_score_frequencies

defaultdict(int, {DiceHand(1, score=100): 1})

In [10]:
for i in range(4):
    mcdh.roll()

In [11]:
mcdh.possible_score_frequencies

defaultdict(int,
            {DiceHand(1, score=100): 1,
             None: 3,
             DiceHand(5, score=50): 1,
             DiceHand(5, 5, score=100): 1})

In [13]:
mcdh.roll_obs

5

In [14]:
mcdh.E_any_score

0.4

In [15]:
mcdh.E_score_arithmetic

33.333333333333336